In [1]:
## ------------------------------------------------------------------------------------------
#                                   Dijkstra Algorithm
## ------------------------------------------------------------------------------------------

# Author: Jai Sharma
# Course: ENPM661 - Planning for Autonomous Robots
# Assignment: Project  2 --> Dijkstra Path Planning

# Important Checks 
#       -->  if start node and goal node inputs are integers
#       -->  if start node and goal node is within map
#       -->  if start node and goal node is within obstacle



In [2]:
## ------------------------------------------------------------------------------------------
#                                    Step 0 --> Import Libraries
## ------------------------------------------------------------------------------------------
import numpy as np
import heapq
import time
import cv2
import pygame

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
## ------------------------------------------------------------------------------------------
#                                   Step 1 --> Inputs and Initialization
## ------------------------------------------------------------------------------------------

# Map Size is known
Map_width = 400
Map_height = 250

# Request Input from User 
print("Note: All User Inputs Must be Integers")
print("x coordinates must be Smaller than ", Map_width)
print("y coordinates must be Smaller than ", Map_height, "\n")

Xi = int(input("x coordinate of START node -->  "))
Yi = int(input("y coordinate of START node -->  "))
Xg = int(input("x coordinate of GOAL node -->  "))
Yg = int(input("y coordinate of GOAL node -->  "))

# Initialize Nodes
start = (Xi,Yi) 
goal =  (Xg,Yg) 

# List of all Coordinates on the Map
map_cord = []
for x in range(Map_width + 1): 
    for y in range(Map_height + 1): 
        map_cord.append((x,y)) #appending to the list

# Check if Input Nodes are within Map Limits
if (start in map_cord) and (goal in map_cord):
    print("\n START and GOAL node inputs are within Map Limits")
else:
    print("\n ERROR: START and/or GOAL node inputs are Outside Map Limits")
    print("\n Please enter new node values")

Note: All User Inputs Must be Integers
x coordinates must be Smaller than  400
y coordinates must be Smaller than  250 

x coordinate of START node -->  5
y coordinate of START node -->  6
x coordinate of GOAL node -->  200
y coordinate of GOAL node -->  243

 START and GOAL node inputs are within Map Limits


In [4]:
## ------------------------------------------------------------------------------------------
#                                    Step 2 -->  Obstacle Mapping
## ------------------------------------------------------------------------------------------

def Obstacles(map_pts,start,goal):
    obstacle_cord = []
    for pts in map_pts:
        x, y = pts[0], pts[1]
        
        # Circle Obstacle
        if (x > 250): 
            if (y > 125):
                if((x-300)**2 + (y-175)**2 <= (40)**2):
                    obstacle_cord.append((x,y))
        # Hexagon Obstacle
        if x > 165 and x < 235:
            if y > 50 and y < 150:
                if (y - 0.577*x < 24.97): 
                    if (y - 0.55*x > -50):
                        if (y + 0.577*x < 255.829): 
                            if (y + 0.548*x > 169.314):
                                obstacle_cord.append((x,y))
        # Weird Shape Obstacle
        if y > 90 and x < 120:
            if y + 1.232*x > 229.348:
                if y - 0.316*x < 173.608:
                    if (y - 0.857*x > 114.29) or (y + 3.2*x < 436):
                        obstacle_cord.append((x,y))
    
    if start in obstacle_cord:         
        print('Error!! START Node is within the Obstacle Space')

    if goal in obstacle_cord:         
        print('Error!! GOAL Node is within the Obstacle Space')
    
    return(obstacle_cord)

In [5]:
## ------------------------------------------------------------------------------------------
#                                      Step 3 -->  Define Actions
## ------------------------------------------------------------------------------------------

# 8 actions, make 8 functions or sub-functions

def Up(curr_node):   # Action Step to move UP
    new_node_y = curr_node[0] + 1
    new_node = (new_node_y,curr_node[1])
    if new_node[0]>= 0 and new_node[1]>=0:
        return(new_node,True)
    else:
        return(curr_node,False)
    
def Down(curr_node):  # Action Step to move DOWN
    new_node_y = curr_node[0] - 1
    new_node = (new_node_y,curr_node[1])
    if new_node[0]>=0 and new_node[1]>=0:
        return(new_node,True)
    else:
        return(curr_node,False)
    
def Left(curr_node):   # Action Step to move LEFT
    new_node_x = curr_node[1] - 1
    new_node = (curr_node[0],new_node_x)
    if new_node[0]>=0 and new_node[1]>=0:
        return(new_node,True)
    else:
        return(curr_node,False)

def Right(curr_node): # Action Step to move RIGHT
    new_node_x = curr_node[1] + 1
    new_node = (curr_node[0],new_node_x)
    if new_node[0]>=0 and new_node[1]>=0:
        return(new_node,True)
    else:
        return(curr_node,False)

def UpLeft(curr_node): # Action Step to move UP LEFT
    new_node_y = curr_node[0] + 1
    new_node_x = curr_node[1] - 1
    new_node = (new_node_y,new_node_x)
    if new_node[0]>=0 and new_node[1]>=0:
        return(new_node,True)
    else:
        return(curr_node,False)

def UpRight(curr_node): # Action Step to move UP Right
    new_node_y = curr_node[0] + 1
    new_node_x = curr_node[1] + 1
    new_node = (new_node_y,new_node_x)
    if new_node[0]>=0 and new_node[1]>=0:
        return(new_node,True)
    else:
        return(curr_node,False)

def DownLeft(curr_node): # Action Step to move DOWN LEFT
    new_node_y = curr_node[0] - 1
    new_node_x = curr_node[1] - 1
    new_node = (new_node_y,new_node_x)
    if new_node[0]>=0 and new_node[1]>=0:
        return(new_node,True)
    else:
        return(curr_node,False)

def DownRight(curr_node): # Action Step to move DOWN RIGHT
    new_node_y = curr_node[0] - 1
    new_node_x = curr_node[1] + 1
    new_node = (new_node_y,new_node_x)
    if new_node[0]>=0 and new_node[1]>=0:
        return(new_node,True)
    else:
        return(curr_node,False)

# print('No errors in Action Functions')

In [6]:
## ------------------------------------------------------------------------------------------
#                                Step 4 -->  Represent Map as Graph
## ------------------------------------------------------------------------------------------

# building a graph on which the Dijkstra Algorithm will be implemented
# use graph data structure

def map2graph(start,Map_height,Map_width): 
    i, j = start[0], start[1] 
    graph_map={}
    if i < Map_height and j < Map_width:
        
        # if the Node is at a Map corner
        if i == 0 and j == 0: # Bottom Left Corner (0,0)
            graph_map[(i,j)]={(i+1,j+1),(i+1,j),(i,j+1)}
        elif i == Map_height-1 and j ==0: # Bottom Right Corner (400,0)
            graph_map[(i,j)]={(i-1,j),(i-1,j+1),(i,j+1)}
        elif i == Map_height-1 and j==Map_width-1: # Top Right Corner (400,250)
            graph_map[(i,j)]={(i-1,j),(i-1,j-1),(i,j-1)}
        elif j == Map_width-1 and i ==0: # Top Left Corner (0,250)
            graph_map[(i,j)]={(i,j-1),(i+1,j-1),(i+1,j)}
            
        # if the Node is along a Map border
        elif i == Map_height-1 and j!=0 and j!=Map_width-1: # Left Border
            graph_map[(i,j)]={(i,j-1),(i,j+1),(i-1,j-1),(i-1,j),(i-1,j+1)}
        elif j == 0 and i!=0 and i!=Map_height-1: # Bottom Border
            graph_map[(i,j)]={(i-1,j),(i+1,j),(i+1,j+1),(i,j+1),(i-1,j+1)}
        elif i == 0 and j!=0 and j!=Map_width-1: # Right Border 
            graph_map[(i,j)]={(i,j-1),(i,j+1),(i+1,j-1),(i+1,j),(i+1,j+1)}
        elif j == Map_width-1 and i!=0 and i!=Map_height-1: # Top Border
            graph_map[(i,j)]={(i-1,j),(i+1,j),(i+1,j-1),(i,j-1),(i-1,j-1)} 
       
        # for any other Node in map
        else: 
            graph_map[(i,j)]={(i-1,j),(i-1,j+1),(i-1,j-1),(i+1,j-1),(i+1,j),(i+1,j+1),(i,j-1),(i,j+1)}
        
        return(graph_map)
    
    else:
        return('The Start Node is outside the bounds of the Map')


In [7]:
## ------------------------------------------------------------------------------------------
#                                    Step 5 --> Cost of Action Steps
## ------------------------------------------------------------------------------------------

def cost(graph,start):
    cost_dic= {} # let key: node, value: cost
    for key,value in graph.items():
        cost_dic[key]={}
        U,D,R,L = Right(key),Left(key), Up(key), Down(key)
        UpL, UpR, DwL, DwR = UpLeft(key),UpRight(key), DownLeft(key), DownRight(key)
        for n in value:
            # Assign Costs
            if (n == U[0]) or (n == L[0]) or ( n == U[0]) or (n == D[0]):
                cost_dic[key][n] = 1 
            elif (n == UpL[0]) or (n == UpR[0]) or (n == DwL[0]) or (n == DwR[0]):
                cost_dic[key][n]= 1.4
    return(cost_dic)

In [8]:
## ------------------------------------------------------------------------------------------
#                                    Step 6 --> Dijkstra Algorithm
## ------------------------------------------------------------------------------------------

OpenList = {}
ClosedList = []
BackTrackList = {}

def Dijkstra(graph,start):
    OpenList[start]=0
    ClosedList.append(start)
    priority_queue = [(0,start)]
    for node,edge in graph.items(): # retrieve nodes and edges from graph
        OpenList[node]= float('inf')
    Goal_Reached = False
    while len(priority_queue)>0 and Goal_Reached == False:
        dist,node = heapq.heappop(priority_queue)
        for nghbr_node,cost in graph[node].items():
            Cost2Come = dist + cost 
            if Cost2Come < OpenList[nghbr_node]:
                BackTrackList[nghbr_node] = {}
                BackTrackList[nghbr_node][Cost2Come] = node
                OpenList[nghbr_node]= Cost2Come
                heapq.heappush(priority_queue, (Cost2Come, nghbr_node))
                if nghbr_node not in ClosedList:
                    ClosedList.append(nghbr_node)
                    if nghbr_node == goal:
                        print('GOAL Node Reached !!')
                        Goal_Reached = True
                        break
                        
    print("Total Cost to Reach Goal: ",Cost2Come)
                    
    return(OpenList,ClosedList,BackTrackList)     


In [11]:
## ------------------------------------------------------------------------------------------
#                                    Step 7 -->  Back-Tracking 
## ------------------------------------------------------------------------------------------

def BackTrack(BackTrack_Dict,goal,start):#goal is the starting point now and start is the goal point now
    BackTrackList_new = []
    BackTrackList_new.append(start)
    break_loop = False
    while break_loop == False:
        for key,value in BackTrackList.items():
            for _,val in value.items():
                if key==start:
                    if val not in BackTrackList_new:
                        BackTrackList_new.append(start)
                    start = val
                    if val == goal:
                        break_loop = True
                        break
    #returns the backtracked list
    return(BackTrackList_new)


In [12]:
## ------------------------------------------------------------------------------------------
#                                    Step 8 -->  Build Main Function
## ------------------------------------------------------------------------------------------

def main(Map_width,Map_height,start,goal):
    map_pts = []
    for i in range(Map_width + 1): 
        for j in range(Map_height + 1): 
            map_pts.append((i,j)) #appending to the list
    ObstacleList = Obstacles(map_pts,start,goal)
    graph_base = {}
    for i in range(Map_width-1,-1,-1):
        for j in range(Map_height-1,-1,-1):
            graph_map = map2graph((i,j),Map_width,Map_height)
            graph_base[(i,j)] = graph_map[(i,j)]
    for key,value in graph_base.items():
        value_copy = value.copy()
        for cord in value_copy:
            if cord in ObstacleList:
                value.remove(cord) 
    graph_base_copy=graph_base.copy()
    for key,value in graph_base_copy.items():
        if key in cord:
            del graph_base[key]
    graph = cost(graph_base,start)
    OpenList = {}
    BackTrack = {}
    ClosedList = []
    OpenList,ClosedList,BackTrack= Dijkstra(graph,start) 
    OpenList_copy = OpenList.copy()
    for k,v in OpenList_copy.items():
        if OpenList_copy[k] == float('inf'):
            del OpenList[k]
    return(OpenList,ClosedList,BackTrack,ObstacleList)


In [13]:
## ------------------------------------------------------------------------------------------
#                           Step 9 -->  Call Main Functions
## ------------------------------------------------------------------------------------------

print('Running Main Function')

start_time = time.time()

Map_width = 400
Map_height = 250
OpenList,ClosedList,BackTrack_Dict,ObstacleList= main(Map_width,Map_height,start,goal) 

Running Main Function
GOAL Node Reached !!
Total Cost to Reach Goal:  318.9999999999995


In [14]:
## ------------------------------------------------------------------------------------------
#                           Step 10 -->  Call BackTracking Functions
## ------------------------------------------------------------------------------------------ 

BackTrackList = BackTrack(BackTrack_Dict,start,goal)
# print(BackTrackList)

print("Time to Run Algorithm: ",time.time() - start_time, "seconds")

Time to Run Algorithm:  212.0936529636383 seconds


In [22]:
## ------------------------------------------------------------------------------------------
#                                    Step 11 -->  Visualization
## ------------------------------------------------------------------------------------------

# display obstacle map

blank_map = np.zeros((251,401,3),np.uint8) # 251 rows, 401 col

for pts in ObstacleList: 
    x,y = pts[1], pts[0]
    blank_map[(x,y)]=[0,0,255] 

main_map = np.flipud(blank_map)
cv2.imshow('The Map with Obstacles',main_map)
cv2.waitKey(0)
cv2.destroyAllWindows()

backtracked_plot = main_map.copy()
visited_plot = main_map.copy()
visited_plot = cv2.resize(visited_plot,(Map_width,Map_height))


In [29]:
pygame.init()
display_width = 400
display_height = 250
gameDisplay = pygame.display.set_mode((display_width,display_height),pygame.FULLSCREEN)
pygame.display.set_caption('Dijkstra Path Planning Algorithm')
surf = pygame.surfarray.make_surface(visited_plot)
clock = pygame.time.Clock()
loop_break = False

while not loop_break:
    for event in pygame.event.get():   
        if event.type == pygame.QUIT:  
            loop_break = True   

    gameDisplay.fill((0,0,0))
    for path in ClosedList:
        if path not in visited_plot:
            x = path[0]
            y = abs(250-path[1])
            pygame.draw.rect(gameDisplay, (70,70,245), [x,y,1,1])
            pygame.display.flip()

    for path in BackTrackList:    
        pygame.time.wait(5)
        x = path[0]
        y = abs(250-path[1])
        pygame.draw.rect(gameDisplay, (255,255,0), [x,y,1,1])
        pygame.display.flip()

        loop_break = True
    
pygame.quit()

C:\Users\jai10\Anaconda3\envs\pyjai\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [26]:

for path in ClosedList:
    x,y  = path[0], path[1]
    backtracked_plot[(display_height-y,x)]=[245,70,70]
cv2.imshow('Map of all Visited Nodes',backtracked_plot)
cv2.waitKey(0)
cv2.destroyAllWindows()

for path in BackTrackList:
    x,y  = path[0], path[1]
    backtracked_plot[(display_height-y,x)]=[0,255,255] 

cv2.imshow('Map of Dijkstra Algorithm Generated Path', backtracked_plot)
cv2.waitKey(0)
cv2.destroyAllWindows()

print('Program Fully Executed')

Program Fully Executed
